In [ ]:
import os
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_DIR = "outputs/final_model"

assert os.path.isdir(MODEL_DIR), f"Model dir not found: {MODEL_DIR}"
print("Loading from:", MODEL_DIR)

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

id2label = model.config.id2label if hasattr(model.config, "id2label") else None
print("Device:", device)
print("Num labels:", model.config.num_labels)
print("id2label:", id2label)


In [ ]:
def predict(texts):
    """
    Returns list of dicts: {text, pred_label, probs}
    probs order corresponds to label ids (0..num_labels-1).
    """
    if isinstance(texts, str):
        texts = [texts]

    enc = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128
    )
    enc = {k: v.to(device) for k, v in enc.items()}

    with torch.no_grad():
        logits = model(**enc).logits
        probs = torch.softmax(logits, dim=-1).cpu().numpy()

    results = []
    for t, p in zip(texts, probs):
        pred_id = int(np.argmax(p))
        if id2label and str(pred_id) in id2label:
            pred_label = id2label[str(pred_id)]
        elif id2label and pred_id in id2label:
            pred_label = id2label[pred_id]
        else:
            # For our PoC data: 0=negative, 1=positive
            pred_label = "positive" if pred_id == 1 else "negative"

        results.append({
            "text": t,
            "pred_id": pred_id,
            "pred_label": pred_label,
            "probs": [float(x) for x in p],
        })
    return results


In [ ]:
tests = [
    "I loved this product. It was excellent and reliable.",
    "This update was horrible and frustrating.",
    "Better than expected — fantastic experience.",
    "I regret using this feature. It was confusing and messy.",
]

for r in predict(tests):
    neg, pos = r["probs"][0], r["probs"][1]
    print("\nTEXT:", r["text"])
    print("PRED:", r["pred_label"], f"(id={r['pred_id']})")
    print(f"PROBS (neg,pos): ({neg:.4f}, {pos:.4f})")


In [ ]:
text = input("Type a sentence to classify (sentiment): ").strip()
out = predict(text)[0]
neg, pos = out["probs"][0], out["probs"][1]
print("\nPRED:", out["pred_label"], f"(id={out['pred_id']})")
print(f"PROBS (neg,pos): ({neg:.4f}, {pos:.4f})")
